In [ ]:
pip install gymnasium

In [ ]:
from time import sleep
import numpy as np
from IPython.display import clear_output
import gymnasium as gym
from gymnasium.envs.registration import register
import torch
from torch import nn

In [ ]:
#Give colab access to your google drive:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
#Change current directory to folder with MiniPacMan
%cd /gdrive/MyDrive/reinforcement_learning/QNNwBuffer

/gdrive/MyDrive/reinforcement_learning/QNNwBuffer


In [ ]:
#Import MiniPacMan environment class definition
from MiniPacManGym import MiniPacManEnv

In [ ]:
#Register MiniPacMan in your gymnasium environments
register(
    id="MiniPacMan-v0",
    entry_point=MiniPacManEnv,  # Update with your actual module path
    max_episode_steps=20          # You can also set a default here
)

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment MiniPacMan-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
#Create a MiniPacMan gymnasium environment
env = gym.make("MiniPacMan-v0", render_mode="human", frozen_ghost=False)

In [ ]:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6 * 6, 128)  # Flattened board (6x6 → 36)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 4)  # Output 4 possible actions (Up, Down, Left, Right)
        self.activation = nn.ReLU()

    def forward(self, state):
        state = state.view(state.shape[0], -1)  # Flatten from (batch_size, 6, 6) → (batch_size, 36)
        x = self.activation(self.fc1(state))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)  # No activation here; raw Q-values
        return x  # Output shape: (batch_size, 4)

In [ ]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []

    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) >= self.capacity:
            self.buffer.pop(0)
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*[self.buffer[i] for i in indices])
        return torch.stack(states), actions, torch.tensor(rewards), torch.stack(next_states), torch.tensor(dones)

In [ ]:
Q = QNetwork() #initialize a Q network
Q_optimizer = torch.optim.Adam(Q.parameters(), lr=0.01) #feel free to change this

In [ ]:
#set hyperparams -- play with any of these!
gamma=0.999
buffer_size=1000
batch_size=32
num_episodes=10000

RB=ReplayBuffer(buffer_size) #initialize Replay Buffer
epsilon=1 #initialize epsilon

for e in range(num_episodes):
  new_obs,info=env.reset()
  new_obs=torch.tensor(new_obs,dtype=torch.float32)

  done=False
  truncated=False
  steps=0

  while not done and not truncated: #Loop for one episode
    obs=new_obs

    #choose action
    t=np.random.random()
    if t>epsilon:
      action = torch.argmax(Q(new_obs.unsqueeze(0))).item()
    else:
      action=torch.randint(4,(1,)).item()

    #take a step:
    new_obs,reward, done, truncated, info=env.step(action)
    new_obs=torch.tensor(new_obs,dtype=torch.float32)
    RB.push(obs,action,reward,new_obs,done)
    steps+=1

    if len(RB.buffer)>=batch_size:
      states, actions, rewards, next_states, dones=RB.sample(batch_size)
      actions = torch.tensor(actions, dtype=torch.long)
      rewards = torch.tensor(rewards, dtype=torch.float32)
      dones = torch.tensor(dones, dtype=torch.float32)

      preds = Q(states).gather(1, actions.unsqueeze(1)).squeeze()
      targets = rewards + (1 - dones) * gamma * torch.max(Q(next_states))
      loss = ((targets - preds) ** 2).mean()
      Q_optimizer.zero_grad()
      loss.backward()
      Q_optimizer.step()

  #reduce episilon if its not too low:
  epsilon=[epsilon-0.00015 if epsilon > 0.01 else 0.01][0]

  #periodic reporting:
  if e>0 and e%100==0:
    print(f'episode: {e}, steps: {steps}, epislon: {epsilon},win: {reward==10}')



<ipython-input-61-ea10ae6c041f>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards = torch.tensor(rewards, dtype=torch.float32)
<ipython-input-61-ea10ae6c041f>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dones = torch.tensor(dones, dtype=torch.float32)


episode: 100, steps: 5, epislon: 0.9848500000000017,win: False
episode: 200, steps: 1, epislon: 0.9698500000000033,win: False
episode: 300, steps: 1, epislon: 0.954850000000005,win: False
episode: 400, steps: 2, epislon: 0.9398500000000066,win: False
episode: 500, steps: 2, epislon: 0.9248500000000083,win: False
episode: 600, steps: 2, epislon: 0.9098500000000099,win: False
episode: 700, steps: 1, epislon: 0.8948500000000116,win: False
episode: 800, steps: 3, epislon: 0.8798500000000132,win: False
episode: 900, steps: 3, epislon: 0.8648500000000149,win: False
episode: 1000, steps: 3, epislon: 0.8498500000000165,win: False
episode: 1100, steps: 3, epislon: 0.8348500000000182,win: False
episode: 1200, steps: 1, epislon: 0.8198500000000198,win: False
episode: 1300, steps: 1, epislon: 0.8048500000000215,win: False
episode: 1400, steps: 1, epislon: 0.7898500000000231,win: False
episode: 1500, steps: 4, epislon: 0.7748500000000248,win: False
episode: 1600, steps: 4, epislon: 0.75985000000002

In [ ]:
obs, info = env.reset()
done = False
truncated = False

while not done and not truncated:
    env.render()
    obs=torch.tensor(obs,dtype=torch.float32)
    action=torch.argmax(Q(obs.unsqueeze(0))).item()
    obs, reward, done, truncated, info = env.step(action)
    sleep(1)
    clear_output(wait=True)

env.render()
env.close()

xxxxxx
x····x
x····x
x··ᗣ·x
x···ᗧx
xxxxxx

